# 1. 线程的简单使用

In [7]:
(defvar msg "")

MSG

In [8]:
(defun thread-function ()                
    (setf msg "Hello Lisp!"))  

THREAD-FUNCTION

In [9]:
(defvar thread)
(setf thread (sb-thread:make-thread #'thread-function))

#<SB-THREAD:THREAD RUNNING {10068A8483}>

In [10]:
(sb-thread:join-thread thread)

"Hello Lisp!"

In [11]:
msg

"Hello Lisp!"

# 2. 给线程命名

In [12]:
(defvar thread)

THREAD

In [13]:
(setf thread (sb-thread:make-thread #'+ :name "what is the sum of no things adding?") )

#<SB-THREAD:THREAD "what is the sum of no things adding?" RUNNING {1006A204D3}>

In [14]:
(sb-thread:join-thread thread)

0

In [15]:
(sb-thread:thread-name thread)

"what is the sum of no things adding?"

# 3.使用参数调用线程

In [16]:
(defun thread-function-with-args (x y) (+ x y))

THREAD-FUNCTION-WITH-ARGS

In [17]:
(defvar thread)
(setf thread (sb-thread:make-thread #'thread-function-with-args :arguments '(1 2)))

#<SB-THREAD:THREAD FINISHED values: 3 {1006BC9C13}>

In [18]:
(sb-thread:join-thread thread)

3

# 4. join-thread的理解
注意：注意进程在make-thread的时候就已经开始执行了，join-thread是等待进程的结束并获得其返回结果

In [19]:
(defvar a )
(setf a 0)
(defun thread-function-sleep ()                
    (sleep 1)
    (incf a 1)
    )  

THREAD-FUNCTION-SLEEP

In [20]:
(defvar thread1)
(defvar thread2)
(defvar thread3)
(setf thread1 (sb-thread:make-thread #'thread-function-sleep))
(setf thread2 (sb-thread:make-thread #'thread-function-sleep))
(setf thread3 (sb-thread:make-thread #'thread-function-sleep))

#<SB-THREAD:THREAD RUNNING {1006D26A93}>

In [21]:
(sb-thread:join-thread thread1)
(sb-thread:join-thread thread2)
(sb-thread:join-thread thread3)
a

3

# 5. 线程管理相关

## (1) 读取当前线程

In [22]:
sb-thread:*current-thread*

#<SB-THREAD:THREAD "main thread" RUNNING {1002C7CDD3}>

## (2) 获取所有正在执行的线程

In [23]:
(sb-thread:list-all-threads)

(#<SB-THREAD:THREAD "Anonymous thread" RUNNING {100607BBE3}>
 #<SB-THREAD:THREAD "main thread" RUNNING {1002C7CDD3}>)

## (3) 终止正在执行的线程

In [24]:
(sb-thread:list-all-threads)

(#<SB-THREAD:THREAD "Anonymous thread" RUNNING {100607BBE3}>
 #<SB-THREAD:THREAD "main thread" RUNNING {1002C7CDD3}>)

In [25]:
(defvar thread-never-stop)
(setf thread-never-stop (sb-thread:make-thread (lambda () (loop)) :name "never stop"))

#<SB-THREAD:THREAD "never stop" RUNNING {1006F104B3}>

In [26]:
(sb-thread:list-all-threads)

(#<SB-THREAD:THREAD "never stop" RUNNING {1006F104B3}>
 #<SB-THREAD:THREAD "Anonymous thread" RUNNING {100607BBE3}>
 #<SB-THREAD:THREAD "main thread" RUNNING {1002C7CDD3}>)

In [27]:
(sb-thread:terminate-thread thread-never-stop)

NIL

In [28]:
(sb-thread:list-all-threads)

(#<SB-THREAD:THREAD "Anonymous thread" RUNNING {100607BBE3}>
 #<SB-THREAD:THREAD "main thread" RUNNING {1002C7CDD3}>)

# 6. 线程互斥(锁)

In [29]:
(defvar g)
(defvar thread1)
(defvar thread2)

THREAD2

## (1)不使用锁的情况

In [30]:
(setf g 0)

0

In [31]:
(defun thread-function-without-lock()
    (let((l g))
        (sleep 1)
        (setf g (incf l 1))
        l
    )
)

THREAD-FUNCTION-WITHOUT-LOCK

In [32]:

(setf thread1 (sb-thread:make-thread #'thread-function-without-lock))
(setf thread2 (sb-thread:make-thread #'thread-function-without-lock))
(sb-thread:join-thread thread1)
(sb-thread:join-thread thread2)

1

## (2) 使用锁的情况

In [33]:
(setf g 0)

0

In [34]:
(defvar lock)
(setf lock (sb-thread:make-mutex :name "lock" ))

#<SB-THREAD:MUTEX "lock" (free)>

In [35]:
(defun thread-function-with-lock()
    (sb-thread:with-mutex (lock) 
        (let((l g))
            (sleep 1)
            (setf g (incf l 1))
            l
        )
    )
)

THREAD-FUNCTION-WITH-LOCK

In [36]:
(setf thread1 (sb-thread:make-thread #'thread-function-with-lock))
(setf thread2 (sb-thread:make-thread #'thread-function-with-lock))
(sb-thread:join-thread thread1)
(sb-thread:join-thread thread2)

2

# 7. 线程互斥(信号量)

信号量互斥通过signal-semaphore和wait-on-semaphore实现，其实他们相当于p v原语:<br/>
p: signal-semaphore<br/>
v: wait-on-semaphore

## (1)基本使用

In [37]:
(defvar s)
(setf s (sb-thread:make-semaphore :name "g" :count 0))

#S(SB-THREAD:SEMAPHORE
   :NAME "g"
   :%COUNT 0
   :WAITCOUNT 0
   :MUTEX #<SB-THREAD:MUTEX (free) {100743EAB3}>
   :QUEUE #<SB-THREAD:WAITQUEUE  {100743EAE3}>)

In [38]:
(sb-thread:signal-semaphore s)
(sb-thread:signal-semaphore s)
(sb-thread:signal-semaphore s)
(sb-thread:semaphore-count s)

3

In [39]:
(sb-thread:wait-on-semaphore s)
(sb-thread:semaphore-count s)

2

In [40]:
(sb-thread:wait-on-semaphore s)
(sb-thread:semaphore-count s)

1

In [41]:
(sb-thread:wait-on-semaphore s)
(sb-thread:semaphore-count s)

0

In [42]:
;; 注意再次运行wait-on-semaphore将导致阻塞
;; (sb-thread:wait-on-semaphore s)
(sb-thread:semaphore-count s)

0

## (2) 简单例子

In [43]:
(defvar s)
(setf s (sb-thread:make-semaphore :name "g" :count 0))

#S(SB-THREAD:SEMAPHORE
   :NAME "g"
   :%COUNT 0
   :WAITCOUNT 0
   :MUTEX #<SB-THREAD:MUTEX (free) {100766AE23}>
   :QUEUE #<SB-THREAD:WAITQUEUE  {100766AE53}>)

In [44]:
(defun thread-do-working() 
    (sleep 3)
    (sb-thread:signal-semaphore s))

THREAD-DO-WORKING

In [45]:
(sb-thread:make-thread #'thread-do-working)
(sb-thread:wait-on-semaphore s)
(print "finished")


"finished" 

"finished"